#### This code was created referring to the code below.
#### https://www.kaggle.com/daniel601/pytorch-fasterrcnn

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
!pip install bs4

In [3]:
from glob import glob
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from bs4 import BeautifulSoup
import cv2


imgs_dir = list(sorted(glob('D:/PyTorch/kaggle_train_fasterrcnn/dataset_face_mask/images/*.png')))
labels_dir = list(sorted(glob("D:/PyTorch/kaggle_train_fasterrcnn/dataset_face_mask/annotations/*.xml")))

class dataset(Dataset) :
    def __init__(self, imgs, labels) :
        self.imgs = imgs
        self.labels = labels
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])
        self.device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
        
    def __len__(self) :
        return len(self.imgs)
    
    def __getitem__(self, index) :
        x = cv2.imread(self.imgs[index])
        x = self.transform(x).to(self.device)
        
        y = dict()
        with open(self.labels[index]) as f :
            data = f.read()
            soup = BeautifulSoup(data, 'xml')
            data = soup.find_all('object')
            
            box = []
            label = []
            for obj in data :
                xmin = int(obj.find('xmin').text)
                ymin = int(obj.find('ymin').text)
                xmax = int(obj.find('xmax').text)
                ymax = int(obj.find('ymax').text)
                
                label_ = 0
                if obj.find('name').text == 'with_mask' :
                    label_ = 1
                elif obj.find('name').text == 'mask_weared_incorrect' :
                    label_ = 2
                
                box.append([xmin, ymin, xmax, ymax])
                label.append(label_)
                
            box = torch.FloatTensor(box)
            label = torch.IntTensor(label)
            
            y['image_id'] = torch.FloatTensor([index]).to(device)
            y["boxes"] = box.to(device)
            y["labels"] = torch.as_tensor(label, dtype=torch.int64)
            
        return x, y
    
def collate_fn(batch) : 
    return tuple(zip(*batch))

train_data = dataset(imgs_dir, labels_dir)
train_data = DataLoader(train_data, batch_size = 1, shuffle = True,
                       collate_fn = collate_fn)

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


In [4]:
#학습된 모델로 사용 : 학습 시간 아끼자.
#load pre-trained model
import torch.nn as nn
import torch.functional as F
import torchvision.models as models

def get_model(output_shape) :
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, output_shape)
    
    return model

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model = get_model(3).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = 0.001, momentum = 0.9, weight_decay = 0.0001)

In [5]:
#학습
#train
from tqdm.notebook import tqdm

model.train()
num_epoch = 50

# for epoch in range(num_epoch) :
#     epoch_loss = 0
    
#     for imgs, annotations in tqdm(train_data):
#         imgs = list(img.to(device) for img in imgs)
#         annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        
#         predict = model(imgs, annotations)
#         losses = sum(loss for loss in predict.values())        

#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step() 
#         epoch_loss += losses
        
#     print(epoch+1, '/', num_epoch, ' : {:.5f}'.format(epoch_loss))
#     if epoch_loss < 0.1 :
#         print('early stop')
#         break

In [6]:
#out of memory 때문에 학습한 거 잃을까봐 일단 저장
#because of out of memory, save the model first
# torch.save(model.state_dict(), 'model.pt')

In [7]:
#load model
model = get_model(3).to(device)
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [8]:
#결과 확인
#get result
import matplotlib.pyplot as plt
import matplotlib.patches as patches

#배치 사이즈 조정해서 재실행 => out of memory 방지
#Resize the batch-size to prevent out of memory
valid_data = dataset(imgs_dir, labels_dir)
valid_data = DataLoader(valid_data, batch_size = 4, shuffle = True,
                       collate_fn = collate_fn)

def plot_img(img, predict, annotation) :
    fig, ax = plt.subplots(1, 2)
    img = img.cpu().data
    
    ax[0].imshow(img.permute(1, 2, 0)) #rgb, w, h => w, h, rgb
    ax[1].imshow(img.permute(1, 2, 0))
    ax[0].set_title("real")
    ax[1].set_title("predict")
    
    for box in annotation["boxes"] :
        xmin, ymin, xmax, ymax = box
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        ax[0].add_patch(rect)
        
    for box in predict["boxes"] :
        xmin, ymin, xmax, ymax = box
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        ax[1].add_patch(rect)
    plt.show()

model.eval()

with torch.no_grad() :
    for imgs, annotations in valid_data:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        
        preds = model(imgs)
        
        for i in range(3) :
            plot_img(imgs[i], preds[i], annotations[i])
            #plot_img(imgs[i], annotations[i])
        
        break

NameError: name 'dataset' is not defined